In [3]:
#microphone check one two what is this!

In [16]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
from statsmodels.tsa.statespace.sarimax import SARIMAX #pip install statsmodels
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

#from thefuzz import process - not needed anymore

import matplotlib.pyplot as plt
import packaging as p

In [17]:
'''#check counts and names
import pandas as pd, re
from pprint import pprint

xl = pd.ExcelFile("multi-year-station-entry-and-exit-figures.xlsx")

def norm(s): 
    import re
    s = str(s).lower().strip()
    s = re.sub(r"[^a-z0-9]+", " ", s)
    return re.sub(r"\s+", " ", s).strip()

def find_header_row(xl, sheet, scan_rows=60):
    raw = xl.parse(sheet, header=None)
    for i in range(min(scan_rows, len(raw))):
        row = [norm(v) for v in raw.iloc[i].tolist()]
        if any("station" in v for v in row) or (any("entr" in v for v in row) and any("exit" in v for v in row)):
            return i
    return None

s = "2017 Entry & Exit"
hdr = find_header_row(xl, s)
df = xl.parse(s, header=hdr).dropna(how="all").dropna(axis=1, how="all")
pprint([norm(c) for c in df.columns])'''

'#check counts and names\nimport pandas as pd, re\nfrom pprint import pprint\n\nxl = pd.ExcelFile("multi-year-station-entry-and-exit-figures.xlsx")\n\ndef norm(s): \n    import re\n    s = str(s).lower().strip()\n    s = re.sub(r"[^a-z0-9]+", " ", s)\n    return re.sub(r"\\s+", " ", s).strip()\n\ndef find_header_row(xl, sheet, scan_rows=60):\n    raw = xl.parse(sheet, header=None)\n    for i in range(min(scan_rows, len(raw))):\n        row = [norm(v) for v in raw.iloc[i].tolist()]\n        if any("station" in v for v in row) or (any("entr" in v for v in row) and any("exit" in v for v in row)):\n            return i\n    return None\n\ns = "2017 Entry & Exit"\nhdr = find_header_row(xl, s)\ndf = xl.parse(s, header=hdr).dropna(how="all").dropna(axis=1, how="all")\npprint([norm(c) for c in df.columns])'

In [18]:
#paths
RAW_STATION_XLSX   = "multi-year-station-entry-and-exit-figures.xlsx"
RAW_STATIONS_CSV   = "TfL_stations.csv"
RAW_PERF_XLSX      = "tfl-tube-performance.xlsx"

OUT_DIR            = "data/processed"
OUT_STATION_FLOW   = f"{OUT_DIR}/station_flow_clean_basic.csv"
OUT_STATIONS_CLEAN = f"{OUT_DIR}/TfL_stations_clean.csv"
OUT_PERF_CLEAN     = f"{OUT_DIR}/tube_performance_clean_basic.csv"

Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

#cleanin file #1:
def clean_station_workbook(xlsx_path: str) -> pd.DataFrame:
    xl = pd.ExcelFile(xlsx_path)
    frames = []
    
    target_sheets = [s for s in xl.sheet_names if ("entry" in s.lower() and "exit" in s.lower())]
    for s in target_sheets:
        raw = xl.parse(s, header=None) #raw read+ drop empty rows/cols
        raw = raw.dropna(how="all").dropna(axis=1, how="all")
        if raw.empty:
            continue

        df = raw.copy()
        df.columns = [f"col{i}" for i in range(df.shape[1])] #assume first column holds station names
        
        for c in df.columns[1:]:
            df[c] = pd.to_numeric(df[c], errors="coerce") #coerce non first columns to numeric

        num_cols = [c for c in df.columns[1:] if np.issubdtype(df[c].dtype, np.number)]
        if not num_cols:
            continue
        q90 = {c: df[c].quantile(0.90) for c in num_cols}
        total_col = max(q90, key=lambda k: (q90[k] if pd.notna(q90[k]) else -np.inf)) #choose total as numeric column with highest 90th %

        out = pd.DataFrame({
            "station": df["col0"].astype(str).str.strip().str.title(),
            "annual_entries_exits": df[total_col]})

        out = out.replace({"": np.nan, "nan": np.nan})
        out = out[~out["station"].str.contains(r"^total\b|^counts\b|^grand\b", case=False, na=False)]
        out = out.dropna(subset=["station", "annual_entries_exits"]) #drop junk

        if out["annual_entries_exits"].max(skipna=True) < 100_000:
            out["annual_entries_exits"] = out["annual_entries_exits"] * 1_000_000 #millions to absolute
            
        out["daily_avg_passengers"] = out["annual_entries_exits"] / 365.0 #daily average

        m = re.search(r"(20\d{2})", s)
        out["year"] = int(m.group(1)) if m else np.nan
        
        frames.append(out[["year","station","annual_entries_exits","daily_avg_passengers"]])

    if not frames:
        raise RuntimeError("Station workbook: produced no rows. Check file path or sheet names.")

    station_flow = pd.concat(frames, ignore_index=True).drop_duplicates()
    return station_flow

station_flow = clean_station_workbook(RAW_STATION_XLSX)
station_flow.to_csv(OUT_STATION_FLOW, index=False)
print(f"Wrote {OUT_STATION_FLOW}  rows={len(station_flow)}")
display(station_flow.head(10))


#cleanin file #2:
def clean_stations_csv(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    df.columns = [re.sub(r"_+", "_", re.sub(r"[^0-9a-zA-Z]+", "_", c)).strip("_").lower() for c in df.columns] #normalize headers
    
    rename_map = {}
    for cand in ["station", "name", "station_name"]:
        if cand in df.columns:
            rename_map[cand] = "station" #pick common columns
            break
    for cand in ["line", "lines"]:
        if cand in df.columns:
            rename_map[cand] = "line"
            break
    for cand in ["latitude","lat"]:
        if cand in df.columns:
            rename_map[cand] = "lat"
            break
    for cand in ["longitude","lon","lng","long"]:
        if cand in df.columns:
            rename_map[cand] = "lon"
            break
    if rename_map:
        df = df.rename(columns=rename_map)

    if "station" in df.columns:
        df["station"] = df["station"].astype(str).str.strip().str.title()
    if "line" in df.columns:
        df["line"] = df["line"].astype(str).str.strip().str.title()

    for c in ["lat","lon"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce") #lat/lon

    keep = [c for c in ["station","line","lat","lon","borough","zone"] if c in df.columns]
    if keep:
        df = df[keep] #keep useful columns

    df = df.drop_duplicates().dropna(how="all")
    return df

stations_clean = clean_stations_csv(RAW_STATIONS_CSV)
stations_clean.to_csv(OUT_STATIONS_CLEAN, index=False)
print(f"Wrote {OUT_STATIONS_CLEAN}  rows={len(stations_clean)}")
display(stations_clean.head(10))


#cleanin file #3:
def clean_tube_performance(xlsx_path: str) -> pd.DataFrame:
    xl = pd.ExcelFile(xlsx_path)
    frames = []
    for s in xl.sheet_names:
        df = xl.parse(s, header=0).dropna(how="all").dropna(axis=1, how="all")
        if df.shape[1] < 2:
            continue
        
        cols_norm = [re.sub(r"\s+", " ", str(c).strip().lower()) for c in df.columns]
        cmap = {cn: c for cn, c in zip(cols_norm, df.columns)} #normalize headers

        line_col = None
        for key in ["line", "line name", "route"]:
            hit = [cmap[k] for k in list(cmap) if key in k]
            if hit:
                line_col = hit[0]; break
        if line_col is None:
            obj = df.select_dtypes(exclude=[np.number]).columns
            if len(obj) == 0: 
                continue
            line_col = obj[0] #line column?

        date_series = None
        for key in ["period start","start date","date"]:
            hit = [cmap[k] for k in list(cmap) if key in k]
            if hit:
                date_series = pd.to_datetime(df[hit[0]], errors="coerce"); break
        if date_series is None and ("year" in cmap and "month" in cmap):
            y = pd.to_numeric(df[cmap["year"]], errors="coerce")
            m = pd.to_numeric(df[cmap["month"]], errors="coerce")
            date_series = pd.to_datetime(dict(year=y, month=m, day=1), errors="coerce") #date column?

        metric_col = None
        for key in ["excess journey time","ejt","lost customer hours","lch"]:
            hit = [cmap[k] for k in list(cmap) if key in k]
            if hit:
                metric_col = hit[0]; break
        if metric_col is None:
            num = df.select_dtypes(include=[np.number]).columns
            if len(num) == 0: 
                continue
            var = pd.Series({c: pd.to_numeric(df[c], errors="coerce").var(skipna=True) for c in num})
            metric_col = var.sort_values(ascending=False).index[0] #metric column?

        tidy = pd.DataFrame({
            "line": df[line_col].astype(str).str.replace(" line","", case=False).str.title().str.strip(),
            "metric": pd.to_numeric(df[metric_col], errors="coerce")})
        if date_series is not None:
            tidy["date"] = pd.to_datetime(date_series, errors="coerce").dt.to_period("M").dt.to_timestamp()
        else:
            tidy["date"] = pd.NaT

        tidy = tidy.dropna(subset=["line"]).drop_duplicates()
        frames.append(tidy[["line","date","metric"]])

    if not frames:
        raise RuntimeError("Performance workbook: produced no rows. Check file path or sheet contents.")
    merged = pd.concat(frames, ignore_index=True)
    # collapse duplicates on same (line,date) by mean
    merged["date"] = pd.to_datetime(merged["date"], errors="coerce")
    merged = (merged.groupby(["line","date"], as_index=False).agg(metric=("metric","mean")))
    return merged

perf_clean = clean_tube_performance(RAW_PERF_XLSX)
perf_clean.to_csv(OUT_PERF_CLEAN, index=False)
print(f"Wrote {OUT_PERF_CLEAN}  rows={len(perf_clean)}")
display(perf_clean.head(10))

Wrote data/processed/station_flow_clean_basic.csv  rows=2968


,year,station,annual_entries_exits,daily_avg_passengers
0,2017,Nan,253.0,0.693151
1,2017,625,149150.0,408.630137
2,2017,747,147574.0,404.312329
3,2017,669,121364.0,332.504110
4,2017,741,121018.0,331.556164
5,2017,635,113606.0,311.249315
6,2017,634,112437.0,308.046575
7,2017,719,93742.0,256.827397
8,2017,513,112547.0,308.347945
9,2017,852,86663.0,237.432877


Wrote data/processed/TfL_stations_clean.csv  rows=436


,station,line
0,Acton Town,"District, Piccadilly"
1,Barbican,"Circle, Hammersmith & City, Metropolitan"
2,Aldgate,"Circle, Metropolitan"
3,Aldgate East,"District, Hammersmith & City"
4,Alperton,Piccadilly
5,Amersham,Metropolitan
6,Angel,Northern
7,Archway,Northern
8,Arnos Grove,Piccadilly
9,Arsenal,Piccadilly


Wrote data/processed/tube_performance_clean_basic.csv  rows=0


,line,date,metric


In [19]:
#paths and imports
RAW_DIR = Path("data/processed")  #new cleaned CSVs

STATION_FLOW_CSV   = RAW_DIR / "station_flow_clean_basic.csv"
STATIONS_CLEAN_CSV = RAW_DIR / "TfL_stations_clean.csv"
PERF_CLEAN_CSV     = RAW_DIR / "tube_performance_clean_basic.csv"

OUT_DIR = Path("data/processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [20]:
#load + sanity check
station_flow   = pd.read_csv(STATION_FLOW_CSV)
stations_ref   = pd.read_csv(STATIONS_CLEAN_CSV)
perf_clean     = pd.read_csv(PERF_CLEAN_CSV, parse_dates=["date"])

print("station_flow:", station_flow.shape, station_flow.columns.tolist())
print("stations_ref:", stations_ref.shape, stations_ref.columns.tolist())
print("perf_clean:",   perf_clean.shape,   perf_clean.columns.tolist())

display(station_flow.head(3))
display(stations_ref.head(3))
display(perf_clean.head(3))

station_flow: (2968, 4) ['year', 'station', 'annual_entries_exits', 'daily_avg_passengers']
stations_ref: (436, 2) ['station', 'line']
perf_clean: (0, 3) ['line', 'date', 'metric']


,year,station,annual_entries_exits,daily_avg_passengers
0,2017,Nan,253.0,0.693151
1,2017,625,149150.0,408.630137
2,2017,747,147574.0,404.312329


,station,line
0,Acton Town,"District, Piccadilly"
1,Barbican,"Circle, Hammersmith & City, Metropolitan"
2,Aldgate,"Circle, Metropolitan"


,line,date,metric


In [27]:
#monthly performance per line, SARIMA
 
perf = perf_clean.copy()
perf["date"] = pd.to_datetime(perf["date"], errors="coerce").dt.to_period("M").dt.to_timestamp()
perf = (perf.dropna(subset=["line"])
            .groupby(["line","date"], as_index=False)
            .agg(metric=("metric","mean")))
 
line = "Jubilee"  # change to any available line
y = (perf[perf["line"]==line]
       .set_index("date")["metric"]
       .asfreq("MS")
       .interpolate(limit_direction="both"))
 
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
 
split_idx = int(len(y)*0.8) if len(y)>10 else max(1,int(len(y)*0.7))
res = (SARIMAX(y.iloc[:split_idx], order=(1,1,1), seasonal_order=(1,1,1,12),
               enforce_stationarity=False, enforce_invertibility=False)
       .fit(disp=False))
y_hat = res.get_prediction(start=y.index[split_idx], end=y.index[-1]).predicted_mean
 
print({"MAE": mean_absolute_error(y.iloc[split_idx:], y_hat),
       "MAPE": mean_absolute_percentage_error(y.iloc[split_idx:], y_hat)})
 
y.plot(label="actual", figsize=(9,4)); y_hat.plot(label="forecast")
plt.legend(); plt.title(f"{line} – SARIMA (no exogenous)"); plt.show()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [25]:
######### STATION LINE MAPPING
def primary_line(s):
    if pd.isna(s): return np.nan
    part = re.split(r"\s*(?:/|;|,|&|\+|\band\b)\s*", str(s), flags=re.I)[0]  # first token only
    return part.strip().title() if part.strip() else np.nan
 
# ensure we have a 'station' column in both frames (fallback to first col if needed)
if "station" not in station_flow.columns:
    station_flow = station_flow.rename(columns={station_flow.columns[0]: "station"})
if "station" not in stations_ref.columns:
    stations_ref = stations_ref.rename(columns={stations_ref.columns[0]: "station"})
 
# normalise names
station_flow["station"] = station_flow["station"].astype(str).str.strip().str.title()
stations_ref["station"] = stations_ref["station"].astype(str).str.strip().str.title()
 
# find a 'line'/'lines' column; if none, make NaN (merge will still work)
line_col = next((c for c in stations_ref.columns if re.search(r"\blines?\b", str(c), flags=re.I)), None)
stations_ref["line"] = stations_ref[line_col].astype(str) if line_col else np.nan
 
# derive 'primary_line' and build mapping
stations_ref["primary_line"] = stations_ref["line"].apply(primary_line)
stn_map = (stations_ref[["station","primary_line"]]
           .dropna()
           .drop_duplicates()
           .rename(columns={"primary_line":"line"}))
 
# merge to attach one line per station
sf_with_line = station_flow.merge(stn_map, on="station", how="left")
 
# coverage (pure pandas; forced to float for safe formatting)
unmapped = sf_with_line["line"].isna().mean()
coverage = 1.0 - unmapped
print(f"primary-line mapping coverage: {float(coverage):.1%}  (unmapped={float(unmapped):.1%})")
display(sf_with_line.head(5))

primary-line mapping coverage: 0.0%  (unmapped=100.0%)


,year,station,annual_entries_exits,daily_avg_passengers,line
0,2017,Nan,253.0,0.693151,NaN
1,2017,625,149150.0,408.630137,NaN
2,2017,747,147574.0,404.312329,NaN
3,2017,669,121364.0,332.504110,NaN
4,2017,741,121018.0,331.556164,NaN
